In [2]:
import tensorflow as tf
import keras

keras.__version__, tf.__version__

('2.5.0', '2.5.0')

In [3]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import LSTM, Dense, Input, Embedding
# from tensorflow.keras.models import Model
# import os
# os.system('wget https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/preprocessdata.py')
# import preprocessdata as pe        ## It is builted module
# import pandas as pd
# import numpy as np
# import string
# from keras.models import load_model
# import tensorflow as tf
# from sklearn.utils import shuffle
# import unicodedata
# import re
# punctuations = set(string.punctuation)

# digits_rm = str.maketrans('', '', string.digits)
# not_punc = ".?!,|"

# for c in not_punc:
#       punctuations.remove(c)

In [4]:
import numpy as np 
import pandas as pd 

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import tensorflow as tf
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [5]:
# lines=pd.read_csv("/content/drive/MyDrive/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')
url = 'https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv'
lines=pd.read_csv(url, encoding='utf-8')
lines= lines[lines.columns[0:2]]
lines=lines[~pd.isnull(lines['English'])]
lines=lines[~pd.isnull(lines['Hindi'])]
lines=lines.sample(n=25000,random_state=42)
lines.drop_duplicates(inplace=True)
lines.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(24677, 2)

In [6]:
lines['English']=lines['English'].apply(lambda x: x.lower())
lines['Hindi']=lines['Hindi'].apply(lambda x: x.lower())

In [7]:

lines['English']=lines['English'].apply(lambda x: re.sub("'", '', x))
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub("'", '', x))

In [8]:
exclude = set(string.punctuation) 
lines['English']=lines['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Hindi']=lines['Hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [9]:
remove_digits = str.maketrans('', '', digits)
lines['English']=lines['English'].apply(lambda x: x.translate(remove_digits))
lines['Hindi']=lines['Hindi'].apply(lambda x: x.translate(remove_digits))

lines['Hindi'] = lines['Hindi'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

lines['English']=lines['English'].apply(lambda x: x.strip())
lines['Hindi']=lines['Hindi'].apply(lambda x: x.strip())
lines['English']=lines['English'].apply(lambda x: re.sub(" +", " ", x))
lines['Hindi']=lines['Hindi'].apply(lambda x: re.sub(" +", " ", x))

In [10]:
# # input_lang, target_lang, eng_words, hin_words, df = load_dataset()

# # num_encoder_words, num_decoder_words, max_input_sen_len, max_target_sen_len = len(eng_words) + 1, len(hin_words) + 1, max(df['len_eng_words']), max(df['len_hin_words'])
# '''
#         Here I loaded imported prepocessdata.load_dataset
# '''


# url = 'https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv'
# input_dict_token, target_dict_token, reverse_input_dict_token, reverse_target_dict_token, df = pe.load_dataset(url)
# num_encoder_words  = len(input_dict_token) + 1
# num_decoder_words = len(target_dict_token) + 1
# max_input_sen_len = max(df['len_eng_words'])
# max_target_sen_len = max(df['len_hin_words'])

In [11]:

lines['Hindi'] = lines['Hindi'].apply(lambda x : 'START_ '+ x + ' _END')
lines.head()

,English,Hindi
64063,the old man s ghost is obsession with past which rides on the shoulders of present subjecting the welfare of the living to a vain glorification of national culture and an unreasoning dread of change,START_ इस बूढ़े आदमी का भूत और कुछ नहीं अतीत के प्रति उसका ऐसा मोह है जो वर्तमान के कंधे पर सवार है किसी के कल्याण का दमन करते हुए राष्ट्रीय संस्कृति और अकारण परिवर्तन की आशंका से उसे अन्यथा गौरवान्वित करता है _END
125354,as a result of authenticity,START_ सच्चा होने की वजह से _END
5417,special session of lokshabha lower house there is description of it in article but it was established with the th amendment in if at least one tenth of the members of lokshabha brought an proposal in which it is stated to continue national emergency then within days of notice period the session will be called,START_ लोकसभा का विशेष सत्र अनु मे इसका वर्णन है किंतु इसे वें संशोधन से स्थापित किया गया है यदि लोकसभा के कम से कम सद्स्य एक प्रस्ताव लाते है जिसमे राष्ट्रीय आपातकाल को जारी न रखने की बात कही गयी है तो नोटिस देने के दिन के भीतर सत्र बुलाया जायेगा _END
104883,kabir das was one of the greatest poets in indian worship poetry tradition,START_ कबीरदास भारत के भक्ति काव्य परंपरा के महानतम कवियों में से एक थे। _END
40881,in india there is high court whose rights and responsibility is limited as compared to supreme court,START_ भारत में उच्च न्यायालय हैं जिनके अधिकार और उत्तरदायित्व सर्वोच्च न्यायालय की अपेक्षा सीमित हैं। _END


In [12]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = train_test_split(df['English'], df['Hindi'], test_size=0.2 , random_state=42)
# X_train.shape, X_test.shape

In [13]:

all_eng_words=set()
for eng in lines['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['Hindi']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [14]:
lines['length_eng_sentence']=lines['English'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['Hindi'].apply(lambda x:len(x.split(" ")))
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [15]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(30764, 37047)

In [16]:
num_decoder_tokens += 1 
num_encoder_tokens = num_encoder_tokens + 1 

In [17]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [18]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [19]:
lines = shuffle(lines)
lines.head(10)

,English,Hindi,length_eng_sentence,length_hin_sentence
44854,news about nepal,START_ नेपाल की खबर _END,3,5
5238,evolution works that way,START_ क्रमिक विकास ऐसे काम करता है _END,4,8
36167,in india among people birth rate and of them are under age,START_ भारत की जनसंख्या में जन्मों के साथ बढती जनसंख्या के आधे लोग वर्ष से कम आयु के हैं। _END,12,20
8961,this is called the muslim agricultural revolution,START_ यह मुस्लिम कृषि क्रांति कहलाती है। _END,7,8
96674,this also is built with marble and red sandstone,START_ यह भी संगमर्मर एवं लाल बलुआ पत्थर से निर्मित है। _END,9,12
45576,it was printed in monterey i checked and the story began,START_ वह मोंटेरे में छपी थी मैंने जांच करी थी और कहानी ऐसे शुरू हुई थी _END,11,17
23297,constitution has three parts,START_ संविधान के तीन भाग _END,4,6
121643,“”gunmain nirgun nirgun main gun baat chaadi kyun bahinse“”,START_ गुनमैं निरगुन निरगुनमैं गुन बाट छांड़ि क्यूं बहिसे _END,9,10
115427,to leverage our common humanity,START_ अपनी साझा मानवता का फ़ायदा उठाने के लिये। _END,5,10
58571,he was not entitled to entertain those cases where the claim was in dispute,START_ वह उन मामलों को हाथ में नहीं ले सकता था जिनमें दावा विवादग्रस्त हो _END,14,16


In [20]:
X, y = lines['English'], lines['Hindi']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((13104,), (3276,))

In [21]:
# def generate_batch(X, Y, batch_size=1):

#     while True:
#         for j in range(0, len(X), batch_size):
#             encoder_input_data = np.zeros((batch_size, max_input_sen_len), dtype='float32')
#             decoder_input_data = np.zeros((batch_size, max_target_sen_len), dtype='float32')
#             decoder_target_data = np.zeros((batch_size, max_target_sen_len, num_decoder_words), dtype='float32')
#             for i, (input_text, target_text) in enumerate(zip(X[j:j + batch_size], Y[j: j+batch_size])):
#                 for t, w in enumerate(input_text.split()):
#                     encoder_input_data[i, t] = input_dict_token[w]
#                 for t, w in enumerate(target_text.split()):
#                     if t<len(target_text.split())-1:
#                         decoder_input_data[i, t] = target_dict_token[w]
#                     if t > 0:
#                         decoder_target_data[i, t-1, target_dict_token[w]] = 1
#             yield([encoder_input_data, decoder_input_data], decoder_target_data)


In [22]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [72]:
latent_dim=300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_words, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

NameError: ignored

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_words, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [28]:
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2Weights.h5')
from keras.models import load_model

model = load_model('/content/drive/MyDrive/Colab/ResultPNA2.h5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 300)    9249000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    11230500    input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) 

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [24]:
# decoder_model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2DecoderModelW.h5')
# encoder_model.load_weights('/content/drive/MyDrive/Colab/ResultPNA2EncoderModelW.h5')
from keras.models import load_model

encoder_model = load_model('/content/drive/MyDrive/Colab/ResultPNA2EncoderModel.h5')
decoder_model = load_model('/content/drive/MyDrive/Colab/ResultPNA2DecoderModel.h5')

In [25]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_token_index['START_']
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence

In [28]:
# model = load_model('/content/drive/MyDrive/Colab/CopyUntitled4/nmt_model_check.h5')
#from keras.models import load_w
# decoder_model.load_weights('/content/drive/MyDrive/Colab/copyofnmt_model_check.h5')
model = load_model('/content/drive/MyDrive/Colab/CopyUntitled4/nmt_weights22Ep100.h5')

In [ ]:
model.summary()

In [29]:
from sklearn.utils import shuffle
wordLen3 = pd.read_csv('https://raw.githubusercontent.com/prismspeechproject/neural/colab-engines/implementation/TestDataset/WordLen4.csv')
wordLen3 = shuffle(wordLen3)
wordLen3X , wordLen3Y = wordLen3['English'], wordLen3['Hindi']

In [1]:
test_gen = generate_batch(wordLen3X, wordLen3Y, batch_size=1)
k =-1
#model.load_weights(checkpoint_dir)
#checkpoint.restore(model , tf.train.latest_checkpoint(checkpoint_dir))

NameError: ignored

In [26]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [27]:
# k+=1
# (input_seq, actual_output), _ = next(train_gen)
# decoded_sentence = decode_sequence(input_seq)
# print('Input English sentence:', wordLen3X[k:k+1].values[0])
# print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
# print('Predicted Hindi Translation:', decoded_sentence[:-4])
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: the best chooses me ”
Actual Hindi Translation:  सर्वोत्तम मुझे चुनता है 
Predicted Hindi Translation:  ओल्डमैन्स औष्घाणीशाठीऔण् धर्माप्रचारकों आडू किस


In [127]:
# k+=1
# (input_seq, actual_output), _ = next(train_gen)
# decoded_sentence = decode_sequence(input_seq)
# print('Input English sentence:', wordLen3X[k:k+1].values[0])
# print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
# print('Predicted Hindi Translation:', decoded_sentence[:-4])
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: at my normal ”
Actual Hindi Translation:  मेरी आम ” की लम्बाई से 
Predicted Hindi Translation:  सहज कृष्णपुरम संशोधनोंस्थानापन्न क़्यों राजशाह


In [30]:
# k+=1
# (input_seq, actual_output), _ = next(test_gen)
# decoded_sentence = decode_sequence(input_seq)
# print('Input English sentence:', wordLen3X[k:k+1].values[0])
# print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
# print('Predicted Hindi Translation:', decoded_sentence[:-4])
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and my body had clearly changed and matt mussolina
Actual Hindi Translation:  और ज़ाहिर है कि मेरा शरीर बदल रहा था और मैट मुसोलिना 
Predicted Hindi Translation:  इष्टतम तारो गई। ज़मीन क़्यों प्राथमिकताएं ग्रीष्मक


In [31]:
# k+=1
# (input_seq, actual_output), _ = next(test_gen)
# decoded_sentence = decode_sequence(input_seq)
# print('Input English sentence:', wordLen3X[k:k+1].values[0])
# print('Actual Hindi Translation:', wordLen3Y[k:k+1].values[0][6:-4])
# print('Predicted Hindi Translation:', decoded_sentence[:-4])

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: tools to translate hindi in firefox
Actual Hindi Translation:  फायरफॉक्स में हिन्दी टाइपिंग के औजार 
Predicted Hindi Translation:  इलियास कारखाना मौर्यों मजेदार कॉर्नर हाइड्रोzzफि


In [32]:
from nltk.translate.bleu_score import corpus_bleu

In [33]:
print('BLEU-1: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(1.0, 0, 0, 0)))
print('BLEU-2: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0.33, 0.33, 0.33, 0)))
print('BLEU-4: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.000000
BLEU-2: 0.000000
BLEU-3: 0.000000
BLEU-4: 0.000000


In [34]:
print('Individual 1-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % corpus_bleu([actual_output], [decoded_sentence], weights=(0, 0, 0, 1)))

Individual 1-gram: 0.000000
Individual 2-gram: 0.000000
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000


In [35]:
encoder_model.get_weights()

[array([[-0.03819279, -0.0322782 , -0.03939082, ...,  0.00416565,
          0.02199462, -0.04040111],
        [-0.24462663,  0.07550556, -0.02188983, ..., -0.05438171,
         -0.05992344, -0.14644411],
        [-0.02854334,  0.01091386,  0.06664548, ..., -0.00438381,
          0.09545403, -0.08324859],
        ...,
        [-0.01087225,  0.05336149,  0.09070357, ..., -0.01562318,
         -0.02250707, -0.18338294],
        [-0.1053583 ,  0.04338017, -0.00409952, ...,  0.06310016,
          0.02308963,  0.11768317],
        [-0.00641337, -0.00235736, -0.00291197, ...,  0.06093942,
         -0.07201329, -0.03548596]], dtype=float32),
 array([[-2.3507608e-01, -5.9964079e-02,  3.3216095e-01, ...,
          1.4458965e-01,  2.2424659e-01, -2.0234863e-01],
        [ 1.1874749e-01, -1.2757336e-01,  2.1041945e-02, ...,
          2.0338273e-05, -2.1128204e-01,  2.9830477e-01],
        [-7.7597684e-01, -2.7464086e-01, -8.0915622e-02, ...,
         -2.5459874e-01, -1.7398483e-01, -1.4230114e-01]